In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/global-modellingdatasetdistance/Global_ModellingDataset_distance-processed.csv


In [2]:
!pip install openpyxl

In [3]:
#import dataset 
df = pd.read_csv(r'../input/global-modellingdatasetdistance/Global_ModellingDataset_distance-processed.csv', sep=';')


## Dataset Preparation

In [4]:
pd.set_option('display.max_columns', None)
df.head()

,FID,wkd_day_value,wnd_day_value,wkd_night_value,wnd_night_value,mean_value_NO2,radiation,nightlight_450,nightlight_4950,nightlight_3150,nightlight_900,elevation,industry_1000,industry_100,industry_25,industry_3000,industry_300,industry_5000,industry_500,industry_50,population_1000,population_3000,population_5000,road_class_1_1000,road_class_1_100,road_class_1_25,road_class_1_3000,road_class_1_300,road_class_1_5000,road_class_1_500,road_class_1_50,road_class_2_1000,road_class_2_100,road_class_2_25,road_class_2_3000,road_class_2_300,road_class_2_5000,road_class_2_500,road_class_2_50,road_class_3_1000,road_class_3_100,road_class_3_25,road_class_3_3000,road_class_3_300,road_class_3_5000,road_class_3_500,road_class_3_50,temperature_2m_10,temperature_2m_11,temperature_2m_12,temperature_2m_1,temperature_2m_2,temperature_2m_3,temperature_2m_4,temperature_2m_5,temperature_2m_6,temperature_2m_7,temperature_2m_8,temperature_2m_9,TROP_2018,wind_speed_10m_10,wind_speed_10m_11,wind_speed_10m_12,wind_speed_10m_1,wind_speed_10m_2,wind_speed_10m_3,wind_speed_10m_4,wind_speed_10m_5,wind_speed_10m_6,wind_speed_10m_7,wind_speed_10m_8,wind_speed_10m_9,trop_mean_filt_2019,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,BldDen100,BldDen500,BldDen1000,NDVI,trafBuf25,trafBuf50,trafBuf100,trafBuf400,trafBuf800,dist,Longitude,Latitude
0,0,30.711957,24.263388,30.438486,28.947476,29.250500,3.192437,22.585377,8.363607,13.402575,21.461321,473.936554,330625,0,0,4466250,39375,6101250,92500,0,8123.319336,54316.64453,95795.42188,0.00000,0.0,0.0,25200.30273,0.000000,49436.99219,0.000000,0.0,0.000000,0.000000,0.000000,7335.407715,0.000000,28671.41016,0.000000,0.000000,31902.113280,313.874695,67.135361,203502.42190,4167.064941,373058.6875,8838.760742,180.451675,10.528473,3.994838,0.855892,-4.079916,2.809515,6.935457,7.397969,13.671423,18.533785,18.813995,18.982145,12.485922,365.725586,2.585911,2.669442,2.989511,2.531975,2.521507,2.995023,2.301084,1.968707,2.190526,2.260399,1.821999,1.986542,0.000058,32.444660,30.905764,45.278419,86.813194,76.644829,84.025391,139.684372,131.750168,62.195648,54.812149,82.809517,62.866955,1.000663,0.910937,0.882346,2310,0.000000,0.000000,0.000000,0.000000,0.000000,1210.352092,10.008293,48.397079
1,1,36.081548,24.658060,32.506374,26.049035,32.098936,2.766697,23.546194,24.685654,22.862799,23.769573,6.617548,936250,0,0,6596250,19375,16483125,131875,0,9558.207031,76786.28125,215604.75000,11369.98047,0.0,0.0,34764.07031,1696.450439,62711.52344,2980.910156,0.0,0.000000,0.000000,0.000000,8974.225586,0.000000,47182.19922,0.000000,0.000000,14582.227540,440.061707,0.000000,99596.04688,1850.278076,313205.7188,5933.034668,58.030884,12.299079,6.153886,3.775151,0.131844,2.825560,6.828553,7.554647,13.972361,17.087158,17.238230,17.520105,13.991841,470.292419,3.869850,3.246683,3.802317,2.791856,3.617435,3.091028,3.455901,2.932038,3.273580,2.649281,2.828152,2.777970,0.000081,64.174301,52.970375,69.828476,50.332016,78.252930,146.870132,105.720657,64.659004,70.474373,117.034981,84.549225,82.943985,0.978245,0.744309,0.658124,2518,0.000000,0.000000,0.000000,3519.916667,3335.471469,126.920051,10.021227,53.523262
2,2,74.562843,51.863782,42.898774,32.600595,58.703744,2.750459,18.290140,15.945211,16.493486,18.070522,18.593605,2500,0,0,343125,0,1212500,0,0,10471.541020,88439.60938,222648.40630,0.00000,0.0,0.0,0.00000,0.000000,11057.72949,0.000000,0.0,5136.330566,270.224976,46.359188,30608.843750,1437.363037,84130.39844,2333.146484,108.745453,34317.210940,157.915741,10.258605,265563.71880,2016.838379,646760.6250,6662.844727,52.477299,12.236689,6.117405,3.744394,0.196013,2.759445,6.723662,7.453573,13.866946,16.986099,17.125896,17.437189,13.936767,457.479950,3.886321,3.264446,3.828975,2.811795,3.632486,3.095585,3.479126,2.962678,3.304003,2.660490,2.845692,2.803313,0.000076,63.257851,53.338516,68.428001,50.577240,76.791069,146.810501,105.269150,63.642395,72.389526,120.899490,85.652794,83.520721,0.914638,0.847111,0.803237,3216,674.791667,674.791667,674.791667,674.

In [5]:
#replace NA with 0
df=df.fillna(0)

#create unique identifier
df['FID'] = range(1,len(df)+1, 1)

## Create spatial groups

### Urban

In [6]:
df['Spachar'] = np.where(df['population_1000'].gt(df['population_1000'].quantile(0.75)) & (df['road_class_1_100'].gt(0) | df['road_class_2_100'].gt(0) | df['road_class_3_100'].gt(df['road_class_3_100'].quantile(0.75))), 1, 0)

### Low population

In [7]:
df['Spachar'] = np.where(df['population_1000'].lt(df['population_1000'].quantile(0.75)) & (df['road_class_1_100'].gt(0) | df['road_class_2_100'].gt(0)  | df['road_class_3_100'].gt(df['road_class_3_100'].quantile(0.75))), 2, df['Spachar'])

### Far from road

In [8]:
df['Spachar'] = np.where((df['Spachar']  == 1) | (df['Spachar'] == 2), df['Spachar'], 3)

In [9]:
# df.head(20)

### filter to limit no. columns for analyzing purposes 

In [10]:
df_sel = df[['Longitude', 'Latitude', 'mean_value_NO2', 'population_1000', 'road_class_1_100', 'road_class_2_100', 'road_class_3_100', 'Spachar']]

In [11]:
#create different datasets so that NO2 distribution per spatial character can be easily examined

Urb = df_sel.loc[df_sel['Spachar'] == 1]
Lowpop = df_sel.loc[df_sel['Spachar'] == 2]
FFR = df_sel.loc[df_sel['Spachar'] == 3]

In [12]:
Urb['mean_value_NO2'].describe()

count    85.000000
mean     38.864759
std      13.065025
min      15.767615
25%      28.172055
50%      38.076120
75%      47.923019
max      78.881718
Name: mean_value_NO2, dtype: float64

In [13]:
Lowpop['mean_value_NO2'].describe()

count    138.000000
mean      27.601430
std        9.768965
min        7.872379
25%       19.872062
50%       26.875778
75%       34.407483
max       56.705580
Name: mean_value_NO2, dtype: float64

In [14]:
FFR['mean_value_NO2'].describe()

count    259.000000
mean      16.652500
std        8.340674
min        2.121931
25%       10.330954
50%       15.891631
75%       22.517860
max       48.886509
Name: mean_value_NO2, dtype: float64

#### Remove unnecessary variables before doing model evaluation per group (spatial character)

In [15]:
#remove unique idenifier & geodata
df_model = df.drop(['Longitude', 'Latitude', 'wkd_day_value', 'wnd_day_value','wkd_night_value', 'wnd_night_value', 'Spachar'], axis=1)

In [16]:
df_model = df_model[['FID', 'mean_value_NO2', 'nightlight_450', 'nightlight_3150', 'population_1000', 'population_3000', 'road_class_2_25', 'road_class_3_300', 'road_class_3_3000', 'trop_mean_filt_2019', 'BldDen100', 'NDVI', 'trafBuf25', 'trafBuf50']]